<a href="https://colab.research.google.com/github/andres-merino/AprendizajeAutomaticoInicial-05-N0105/blob/main/2-Ejercicios/09-Bosques-Aleatorios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table style="border: none; border-collapse: collapse;">
    <tr>
        <td style="width: 20%; vertical-align: middle; padding-right: 10px;">
            <img src="https://i.imgur.com/nt7hloA.png" width="100">
        </td>
        <td style="width: 2px; text-align: center;">
            <font color="#0030A1" size="7">|</font><br>
            <font color="#0030A1" size="7">|</font>
        </td>
        <td>
            <p style="font-variant: small-caps;"><font color="#0030A1" size="5">
                <b>Facultad de Ciencias Exactas, Naturales y Ambientales</b>
            </font> </p>
            <p style="font-variant: small-caps;"><font color="#0030A1" size="4">
                Aprendizaje Automático Inicial &bull; Bosques Aleatorios
            </font></p>
            <p style="font-style: oblique;"><font color="#0030A1" size="3">
                Emilio Andrade &bull; 2026-18-01
            </font></p>
        </td>  
    </tr>
</table>

---
## <font color='264CC7'> Introducción </font>

En este taller trabajaremos con la base **Global E-Commerce Customer Analytics** que cargaremos desde GitHub, la idea es aplicar **Bosques Aleatorios** para un problema de clasificación, en particular detectar clientes con historial fraudulento usando variables demográficas y de comportamiento, como hay pocos casos positivos en comparación con los negativos vamos a mirar métricas más útiles que solo la accuracy, y también vamos a discutir qué tan bien se sostiene el modelo cuando cambiamos ajustes como el balanceo y la forma de decidir el umbral


Los paquetes necesarios son:

In [1]:
# Paquetes necesarios
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import precision_recall_curve, average_precision_score

---
## <font color='264CC7'> Clasificación </font>


### <font color='264CC7'> Preprocesamiento de datos </font>

Primero necesitas el conjunto de datos. Los datos a utilzar son los seleccionados en la clase anterior.

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
    Carga el conjunto de datos y procésalos:
<ul>
  <li>Muestra algunos datos.</li>
  <li>Muestra una descripción de los datos.</li>
  <li>Escala los datos si es necesario.</li>
</ul>
</div>

In [13]:
url = "https://raw.githubusercontent.com/1EmilioAnd1/CD-Andrade-Emilio/refs/heads/main/synthetic_ecommerce_churn_dataset.csv"
data = pd.read_csv(url)

print("Muestra de datos")
display(data.head())

print("\nDescripcion (solo numericas)")
display(data.describe())

print("\nPorcentaje de faltantes por columna")
display(data.isna().mean().sort_values(ascending=False))

data["customer_since"] = pd.to_datetime(data["customer_since"], errors="coerce")
ref_date = data["customer_since"].max()
data["tenure_days"] = (ref_date - data["customer_since"]).dt.days

print("\nEjemplo de customer_since y tenure_days")
display(data[["customer_since", "tenure_days"]].head())

print("\nDistribucion de la variable objetivo is_fraudulent")
display(data["is_fraudulent"].value_counts())

target = "is_fraudulent"
y = data[target].astype(int)

num_cols = ["age", "avg_order_value", "total_orders", "last_purchase", "email_open_rate", "loyalty_score", "tenure_days", "churn_risk"]
cat_cols = ["gender", "country", "preferred_category"]

X = data[num_cols + cat_cols]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

preprocess = ColumnTransformer(
    transformers=[
        ("num", SimpleImputer(strategy="median"), num_cols),
        ("cat", Pipeline([
            ("imputer", SimpleImputer(strategy="most_frequent")),
            ("ohe", OneHotEncoder(handle_unknown="ignore"))
        ]), cat_cols),
    ]
)

print("\nShapes y proporcion de clase positiva")
print("X_train:", X_train.shape, "X_test:", X_test.shape)
print("Fraude en train:", float(y_train.mean()), "Fraude en test:", float(y_test.mean()))


Muestra de datos


,customer_id,age,gender,country,avg_order_value,total_orders,last_purchase,is_fraudulent,preferred_category,email_open_rate,customer_since,loyalty_score,churn_risk
0,CUST_8270,30,Female,Brazil,101.08,8,176,1,Beauty,25.6,2024-06-05,50,0.20
1,CUST_1860,53,Female,USA,90.39,10,88,0,Electronics,12.3,2024-02-19,37,0.34
2,CUST_6390,73,Male,Australia,83.28,6,203,0,Sports,NaN,2024-04-16,65,0.05
3,CUST_6191,30,Other,Japan,109.90,9,346,1,Electronics,42.9,2020-07-08,93,0.19
4,CUST_6734,29,Female,Canada,269.38,16,342,0,Fashion,5.3,2025-04-09,79,0.15



Descripcion (solo numericas)


,age,avg_order_value,total_orders,last_purchase,is_fraudulent,email_open_rate,loyalty_score,churn_risk
count,5000.000000,4750.000000,5000.000000,5000.000000,5000.000000,4750.000000,5000.000000,5000.000000
mean,48.163200,108.442857,10.027000,180.073200,0.025800,50.714842,50.039400,0.284484
std,17.880797,69.265559,3.163838,104.926518,0.158554,29.098706,28.832151,0.159690
min,18.000000,10.660000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,33.000000,57.805000,8.000000,89.000000,0.000000,25.225000,25.000000,0.160000
50%,48.000000,93.190000,10.000000,178.000000,0.000000,50.950000,50.000000,0.260000
75%,64.000000,142.197500,12.000000,270.000000,0.000000,76.800000,75.000000,0.390000
max,79.000000,555.460000,23.000000,364.000000,1.000000,100.000000,99.000000,0.900000



Porcentaje de faltantes por columna


...


Ejemplo de customer_since y tenure_days


,customer_since,tenure_days
0,2024-06-05,388
1,2024-02-19,495
2,2024-04-16,438
3,2020-07-08,1816
4,2025-04-09,80



Distribucion de la variable objetivo is_fraudulent


,count
is_fraudulent,
0,4871
1,129



Shapes y proporcion de clase positiva
X_train: (4000, 11) X_test: (1000, 11)
Fraude en train: 0.02575 Fraude en test: 0.026


En esta parte ya cargamos la base desde GitHub y verificamos que esté lista para trabajar, primero mostramos una muestra para confirmar que las columnas tienen sentido y que la variable objetivo is_fraudulent está en formato 0 y 1, después revisamos la descripción de las variables numéricas y se nota que age va de 18 a 79, total_orders llega hasta 23 y last_purchase llega hasta 364 días, también se ve que avg_order_value y email_open_rate tienen faltantes cerca del 5% y por eso más adelante vamos a imputar en vez de borrar filas

Luego convertimos customer_since a fecha y creamos tenure_days, esto ayuda porque resume cuánto tiempo lleva el cliente en la plataforma y puede aportar información al modelo, al revisar la distribución de is_fraudulent se confirma un desbalance fuerte, solo 129 casos positivos frente a 4871 negativos, por eso no vamos a quedarnos con accuracy como única métrica porque puede verse alta aunque el modelo no detecte fraudes

Finalmente se separan variables numéricas y categóricas y se hace el split train test con stratify, los tamaños quedan en 4000 y 1000 y la proporción de clase positiva se mantiene casi igual en ambos conjuntos, esto es importante porque así la evaluación en test no queda distorsionada, y con esto ya tenemos el punto de partida correcto para entrenar el bosque y comparar ajustes



### <font color='264CC7'> Entrenamiento del modelo y evaluación </font>
Ahora entrenamos un Bosque Aleatorio como modelo base, la idea es partir con algo estándar y ver cómo se comporta con una clase tan desbalanceada, luego evaluamos en test con matriz de confusión y con métricas de la clase 1, porque ahí es donde se nota si el modelo realmente está encontrando los fraudes o solo está diciendo casi todo como no fraude

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
    Entrena el modelo y evalúalo:
<ul>
  <li>Entrena el modelo.</li>
  <li>Evalúa el modelo con el conjunto de datos de prueba.</li>
  <li>Genera una matriz de confusión y muestra las métricas de evaluación.</li>
</ul>
</div>

In [17]:
rf = RandomForestClassifier(n_estimators=300, random_state=42, n_jobs=-1)

pipe = Pipeline([
    ("prep", preprocess),
    ("model", rf)
])

pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)

cm = confusion_matrix(y_test, y_pred)
rep = classification_report(y_test, y_pred, zero_division=0, output_dict=True)

accuracy = accuracy_score(y_test, y_pred)
precision_1 = rep["1"]["precision"]
recall_1 = rep["1"]["recall"]
f1_1 = rep["1"]["f1-score"]

print("Matriz de confusion")
display(cm)

print("\nMetricas principales")
print("accuracy:", accuracy)
print("precision_1:", precision_1)
print("recall_1:", recall_1)
print("f1_1:", f1_1)

print("\nClassification report")
print(classification_report(y_test, y_pred, zero_division=0))

Matriz de confusion


array([[974,   0],
       [ 26,   0]])


Metricas principales
accuracy: 0.974
precision_1: 0.0
recall_1: 0.0
f1_1: 0.0

Classification report
              precision    recall  f1-score   support

           0       0.97      1.00      0.99       974
           1       0.00      0.00      0.00        26

    accuracy                           0.97      1000
   macro avg       0.49      0.50      0.49      1000
weighted avg       0.95      0.97      0.96      1000



En el modelo base el bosque aleatorio obtiene una accuracy de 0.974, pero cuando vemos la matriz de confusión se nota el problema real, el modelo predice todo como clase 0 y no detecta ningún caso fraudulento, por eso para la clase 1 la precisión, el recall y el f1 quedan en 0

Esto pasa porque la base está desbalanceada y al modelo le conviene acertar la clase mayoritaria, entonces la accuracy se ve alta aunque el objetivo principal que es encontrar fraudes no se está cumpliendo, por eso en las siguientes pruebas vamos a hacer ajustes para que el modelo le preste más atención a la clase minoritaria y así subir el recall de la clase 1, aunque eso normalmente trae más falsos positivos


In [18]:
rf_bal = RandomForestClassifier(n_estimators=300, random_state=42, n_jobs=-1, class_weight="balanced")

pipe_bal = Pipeline([
    ("prep", preprocess),
    ("model", rf_bal)
])

pipe_bal.fit(X_train, y_train)

y_pred_bal = pipe_bal.predict(X_test)

cm_bal = confusion_matrix(y_test, y_pred_bal)
rep_bal = classification_report(y_test, y_pred_bal, zero_division=0, output_dict=True)

accuracy_bal = accuracy_score(y_test, y_pred_bal)
precision_1_bal = rep_bal["1"]["precision"]
recall_1_bal = rep_bal["1"]["recall"]
f1_1_bal = rep_bal["1"]["f1-score"]

print("Matriz de confusion")
display(cm_bal)

print("\nMetricas principales")
print("accuracy:", accuracy_bal)
print("precision_1:", precision_1_bal)
print("recall_1:", recall_1_bal)
print("f1_1:", f1_1_bal)

print("\nClassification report")
print(classification_report(y_test, y_pred_bal, zero_division=0))

Matriz de confusion


array([[974,   0],
       [ 26,   0]])


Metricas principales
accuracy: 0.974
precision_1: 0.0
recall_1: 0.0
f1_1: 0.0

Classification report
              precision    recall  f1-score   support

           0       0.97      1.00      0.99       974
           1       0.00      0.00      0.00        26

    accuracy                           0.97      1000
   macro avg       0.49      0.50      0.49      1000
weighted avg       0.95      0.97      0.96      1000



Como el bosque sigue clasificando todo como no fraude, ahora no cambiamos el modelo todavía, más bien usamos sus probabilidades y buscamos un umbral que maximice f1, esto normalmente sube el recall de la clase 1 aunque baje la accuracy y aparezcan más falsos positivos, aquí la idea es ver si el modelo tiene algo de señal y solo estaba frenado por el umbral 0.5

In [19]:
proba = pipe_bal.predict_proba(X_test)[:, 1]

ap = average_precision_score(y_test, proba)
prec, rec, thr = precision_recall_curve(y_test, proba)
f1 = (2 * prec * rec) / (prec + rec + 1e-12)

best_idx = int(np.argmax(f1))
best_thr = float(thr[best_idx]) if best_idx < len(thr) else 0.5

y_pred_thr = (proba >= best_thr).astype(int)

cm_thr = confusion_matrix(y_test, y_pred_thr)
rep_thr = classification_report(y_test, y_pred_thr, zero_division=0, output_dict=True)

accuracy_thr = accuracy_score(y_test, y_pred_thr)
precision_1_thr = rep_thr["1"]["precision"]
recall_1_thr = rep_thr["1"]["recall"]
f1_1_thr = rep_thr["1"]["f1-score"]

print("Average precision:", ap)
print("Umbral elegido:", best_thr)

print("\nResumen de probabilidades")
print("min:", float(proba.min()), "max:", float(proba.max()))
print("p90:", float(np.quantile(proba, 0.9)), "p95:", float(np.quantile(proba, 0.95)), "p99:", float(np.quantile(proba, 0.99)))

print("\nMatriz de confusion")
display(cm_thr)

print("\nMetricas principales")
print("accuracy:", accuracy_thr)
print("precision_1:", precision_1_thr)
print("recall_1:", recall_1_thr)
print("f1_1:", f1_1_thr)

print("\nClassification report")
print(classification_report(y_test, y_pred_thr, zero_division=0))


Average precision: 0.030482663821244625
Umbral elegido: 0.08333333333333333

Resumen de probabilidades
min: 0.0 max: 0.17
p90: 0.05333333333333334 p95: 0.06333333333333334 p99: 0.10003333333333331

Matriz de confusion


array([[954,  20],
       [ 24,   2]])


Metricas principales
accuracy: 0.956
precision_1: 0.09090909090909091
recall_1: 0.07692307692307693
f1_1: 0.08333333333333333

Classification report
              precision    recall  f1-score   support

           0       0.98      0.98      0.98       974
           1       0.09      0.08      0.08        26

    accuracy                           0.96      1000
   macro avg       0.53      0.53      0.53      1000
weighted avg       0.95      0.96      0.95      1000



Con el ajuste de umbral se ve que el problema no era solo el 0.5, las probabilidades del modelo para la clase 1 son muy bajas, el máximo llega a 0.17 y por eso el modelo base y el balanced terminaban prediciendo todo como 0

Al buscar un umbral por f1 se obtiene 0.0833 y recién ahí aparecen positivos, aun así el rendimiento sigue siendo limitado, la matriz de confusión muestra 2 fraudes detectados de 26 y 20 falsos positivos, esto deja un recall de 0.077 y una precisión de 0.091, además el average precision es 0.0305 y eso sugiere que la capacidad de ordenar bien los casos fraudulentos es baja con las variables actuales, entonces el modelo mejora un poco pero todavía no es un detector confiable


In [20]:
rf_reg = RandomForestClassifier(
    n_estimators=500,
    random_state=42,
    n_jobs=-1,
    class_weight="balanced",
    max_depth=8,
    min_samples_split=20,
    min_samples_leaf=10
)

pipe_reg = Pipeline([
    ("prep", preprocess),
    ("model", rf_reg)
])

pipe_reg.fit(X_train, y_train)

proba_reg = pipe_reg.predict_proba(X_test)[:, 1]
ap_reg = average_precision_score(y_test, proba_reg)

prec_reg, rec_reg, thr_reg = precision_recall_curve(y_test, proba_reg)
f1_reg = (2 * prec_reg * rec_reg) / (prec_reg + rec_reg + 1e-12)

best_idx_reg = int(np.argmax(f1_reg))
best_thr_reg = float(thr_reg[best_idx_reg]) if best_idx_reg < len(thr_reg) else 0.5

y_pred_reg = (proba_reg >= best_thr_reg).astype(int)

cm_reg = confusion_matrix(y_test, y_pred_reg)
rep_reg = classification_report(y_test, y_pred_reg, zero_division=0, output_dict=True)

accuracy_reg = accuracy_score(y_test, y_pred_reg)
precision_1_reg = rep_reg["1"]["precision"]
recall_1_reg = rep_reg["1"]["recall"]
f1_1_reg = rep_reg["1"]["f1-score"]

print("Average precision:", ap_reg)
print("Umbral elegido:", best_thr_reg)

print("\nResumen de probabilidades")
print("min:", float(proba_reg.min()), "max:", float(proba_reg.max()))
print("p90:", float(np.quantile(proba_reg, 0.9)), "p95:", float(np.quantile(proba_reg, 0.95)), "p99:", float(np.quantile(proba_reg, 0.99)))

print("\nMatriz de confusion")
display(cm_reg)

print("\nMetricas principales")
print("accuracy:", accuracy_reg)
print("precision_1:", precision_1_reg)
print("recall_1:", recall_1_reg)
print("f1_1:", f1_1_reg)

print("\nClassification report")
print(classification_report(y_test, y_pred_reg, zero_division=0))

Average precision: 0.031018855361616603
Umbral elegido: 0.3964808274339159

Resumen de probabilidades
min: 0.08515608624640784 max: 0.5270539517257353
p90: 0.39313704288454276 p95: 0.4218849108930946 p99: 0.48197623962287794

Matriz de confusion


array([[886,  88],
       [ 22,   4]])


Metricas principales
accuracy: 0.89
precision_1: 0.043478260869565216
recall_1: 0.15384615384615385
f1_1: 0.06779661016949153

Classification report
              precision    recall  f1-score   support

           0       0.98      0.91      0.94       974
           1       0.04      0.15      0.07        26

    accuracy                           0.89      1000
   macro avg       0.51      0.53      0.50      1000
weighted avg       0.95      0.89      0.92      1000



### <font color='264CC7'> Prueba sin churn_risk </font>

Ahora repetimos el entrenamiento quitando churn_risk, la idea es ver si el modelo estaba apoyándose demasiado en esa variable o si el rendimiento se mantiene parecido, esto ayuda a justificar mejor qué variables realmente aportan y también nos deja una conclusión más limpia, porque churn_risk es una probabilidad de churn y no necesariamente un indicador directo de fraude


In [21]:
num_cols2 = ["age", "avg_order_value", "total_orders", "last_purchase", "email_open_rate", "loyalty_score", "tenure_days"]
cat_cols2 = ["gender", "country", "preferred_category"]

X2 = data[num_cols2 + cat_cols2]
y2 = data["is_fraudulent"].astype(int)

X2_train, X2_test, y2_train, y2_test = train_test_split(
    X2, y2, test_size=0.2, random_state=42, stratify=y2
)

preprocess2 = ColumnTransformer(
    transformers=[
        ("num", SimpleImputer(strategy="median"), num_cols2),
        ("cat", Pipeline([
            ("imputer", SimpleImputer(strategy="most_frequent")),
            ("ohe", OneHotEncoder(handle_unknown="ignore"))
        ]), cat_cols2),
    ]
)

rf_no_churn = RandomForestClassifier(
    n_estimators=500,
    random_state=42,
    n_jobs=-1,
    class_weight="balanced",
    max_depth=8,
    min_samples_split=20,
    min_samples_leaf=10
)

pipe_no_churn = Pipeline([
    ("prep", preprocess2),
    ("model", rf_no_churn)
])

pipe_no_churn.fit(X2_train, y2_train)

proba_no_churn = pipe_no_churn.predict_proba(X2_test)[:, 1]
ap_no_churn = average_precision_score(y2_test, proba_no_churn)

prec_nc, rec_nc, thr_nc = precision_recall_curve(y2_test, proba_no_churn)
f1_nc = (2 * prec_nc * rec_nc) / (prec_nc + rec_nc + 1e-12)

best_idx_nc = int(np.argmax(f1_nc))
best_thr_nc = float(thr_nc[best_idx_nc]) if best_idx_nc < len(thr_nc) else 0.5

y_pred_no_churn = (proba_no_churn >= best_thr_nc).astype(int)

cm_no_churn = confusion_matrix(y2_test, y_pred_no_churn)
rep_no_churn = classification_report(y2_test, y_pred_no_churn, zero_division=0, output_dict=True)

accuracy_no_churn = accuracy_score(y2_test, y_pred_no_churn)
precision_1_no_churn = rep_no_churn["1"]["precision"]
recall_1_no_churn = rep_no_churn["1"]["recall"]
f1_1_no_churn = rep_no_churn["1"]["f1-score"]

print("Average precision:", ap_no_churn)
print("Umbral elegido:", best_thr_nc)

print("\nResumen de probabilidades")
print("min:", float(proba_no_churn.min()), "max:", float(proba_no_churn.max()))
print("p90:", float(np.quantile(proba_no_churn, 0.9)), "p95:", float(np.quantile(proba_no_churn, 0.95)), "p99:", float(np.quantile(proba_no_churn, 0.99)))

print("\nMatriz de confusion")
display(cm_no_churn)

print("\nMetricas principales")
print("accuracy:", accuracy_no_churn)
print("precision_1:", precision_1_no_churn)
print("recall_1:", recall_1_no_churn)
print("f1_1:", f1_1_no_churn)

print("\nClassification report")
print(classification_report(y2_test, y_pred_no_churn, zero_division=0))

Average precision: 0.03195308714522719
Umbral elegido: 0.41189491494055047

Resumen de probabilidades
min: 0.0970149167953801 max: 0.5478151740299609
p90: 0.40509032991964283 p95: 0.43280371649306637 p99: 0.4741990583864744

Matriz de confusion


array([[892,  82],
       [ 21,   5]])


Metricas principales
accuracy: 0.897
precision_1: 0.05747126436781609
recall_1: 0.19230769230769232
f1_1: 0.08849557522123894

Classification report
              precision    recall  f1-score   support

           0       0.98      0.92      0.95       974
           1       0.06      0.19      0.09        26

    accuracy                           0.90      1000
   macro avg       0.52      0.55      0.52      1000
weighted avg       0.95      0.90      0.92      1000



Al quitar churn_risk el modelo no empeora, de hecho mejora un poco en la clase 1, el recall sube a 0.192 y el f1 a 0.088, la matriz de confusión muestra 5 fraudes detectados de 26 y 82 falsos positivos, entonces el modelo sigue siendo sensible a generar alertas que no son fraude pero al menos recupera más casos reales que las versiones anteriores

Esto sugiere que churn_risk no aporta como predictor de fraude en este ejercicio, y que incluso puede meter ruido porque fue pensado para churn y no para fraude, además el average precision queda en 0.032 y aunque sigue siendo bajo, es el valor más alto de las pruebas, por lo tanto para este taller la mejor variante es el bosque balanceado y regularizado sin churn_risk con ajuste de umbral


In [24]:
resumen = pd.DataFrame([
    {"modelo": "Base (umbral 0.5)", "ap": None, "accuracy": 0.974, "precision_1": 0.0, "recall_1": 0.0, "f1_1": 0.0},
    {"modelo": "Balanced + umbral (F1)", "ap": 0.030482663821244625, "accuracy": 0.956, "precision_1": 0.09090909090909091, "recall_1": 0.07692307692307693, "f1_1": 0.08333333333333333},
    {"modelo": "Regularizado + umbral (F1)", "ap": 0.031018855361616603, "accuracy": 0.89, "precision_1": 0.043478260869565216, "recall_1": 0.15384615384615385, "f1_1": 0.06779661016949153},
    {"modelo": "Sin churn_risk + umbral (F1)", "ap": 0.03195308714522719, "accuracy": 0.897, "precision_1": 0.05747126436781609, "recall_1": 0.19230769230769232, "f1_1": 0.08849557522123894},
])

display(resumen)

,modelo,ap,accuracy,precision_1,recall_1,f1_1
0,Base (umbral 0.5),NaN,0.974,0.000000,0.000000,0.000000
1,Balanced + umbral (F1),0.030483,0.956,0.090909,0.076923,0.083333
2,Regularizado + umbral (F1),0.031019,0.890,0.043478,0.153846,0.067797
3,Sin churn_risk + umbral (F1),0.031953,0.897,0.057471,0.192308,0.088496


## <font color='264CC7'> Conclusiones </font>

En este taller se entrenó un Bosque Aleatorio para detectar is_fraudulent y se comprobó que la base está muy desbalanceada, por eso el modelo base con umbral 0.5 logra una accuracy alta de 0.974 pero no detecta ningún fraude, en la práctica eso no sirve para el objetivo del ejercicio

Cuando se aplicó balanceo y se ajustó el umbral usando f1 el modelo empezó a marcar positivos, pero el rendimiento sigue limitado, el average precision se mantiene cerca de 0.03 en todas las variantes y eso sugiere que la separación entre fraude y no fraude es baja con las variables disponibles, aun así se ve el trade off, al hacer el modelo más sensible sube el recall pero aparecen falsos positivos y la precisión baja

La mejor variante de las probadas fue quitar churn_risk y mantener el bosque balanceado y regularizado con ajuste de umbral, porque alcanza el recall más alto de 0.192 y el f1 más alto de 0.088 dentro del taller, además deja una idea clara, churn_risk no aporta como predictor de fraude y puede meter ruido porque fue pensado para churn, en resumen el flujo queda bien armado y defendible, pero también muestra que para mejorar de verdad harían falta variables más directas sobre fraude o un enfoque con costos y validación más fuerte